# Python抓取动态网页数据


作者：上海交通大学 安泰经济与管理学院 周志中

2018-09-15, updated on 2018-09-29


## 使用Selenium + PhantomJS 抓取动态网页数据

如果网页是HTML静态生成的内容，那么直接从HTML源码中就能找到看到的数据和内容，然而并不是所有的网页都是这样的。有一些网站的内容由前端的Java Script动态生成，由于呈现在网页上的内容是由Java Script生成而来，我们能够在浏览器上看得到，但是在HTML源码中却发现不了。例如[微博财经影响力排行榜](https://v6.bang.weibo.com/czv/caijing)，排行榜中的内容是JS生成的。在Firefox当中审查元素发现，排行榜中的微博名保存在标签`<h3>`之下，但使用下面抓取静态网页数据的方法尝试提取标签`<h3>`中的数据，结果失败。

In [2]:
from bs4 import BeautifulSoup
import urllib.request

url='https://v6.bang.weibo.com/czv/caijing/'
resp=urllib.request.urlopen(url)
html=resp.read().decode("utf-8")
bs = BeautifulSoup(html, "lxml")
contents = bs.find_all('h3')
contents

[]

使用Selenium+Firefox抓取也失败。

In [3]:
from selenium import webdriver

driver = webdriver.Firefox();
driver.get("https://v6.bang.weibo.com/czv/caijing");

elems = driver.find_elements_by_tag_name("h3");
print(len(elems))

0


抓取动态网页数据我们需要使用Selenium + PhantomJS。PhantomJS是基于Webkit的无界面（headless）浏览器。需要从[官方网站](phantomjs.org)下载，再将bin目录添加到环境变量Path中。如果我们把Selenium和PhantomJS结合在一起，就可以运行一个非常强大的网络爬虫，这个爬虫可以处理Java Script、Cookie、headers，以及任何我们真实用户需要做的事情。

In [1]:
from selenium import webdriver

driver = webdriver.PhantomJS();
driver.get("https://v6.bang.weibo.com/czv/caijing");

elems = driver.find_elements_by_tag_name("h3");
print(len(elems))

D:\Applications\Analytics\Anaconda3\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


20


很不幸的是，Selenium将来不再支持PhantomJS，不过现在还可以用，而且我们找到了20个标签`<h3>`中间的元素，恰好是排行榜前20名的微博名。Selenium推荐使用headless Firefox来替换PhantomJS。下面的代码显示，将PhantomJS替换成headless Firefox之后，可以读出动态网页由JS生成的内容，并找到`<h3>`标签下的元素。

In [2]:
options = webdriver.FirefoxOptions()
options.set_headless()
#options.add_argument('-headless')
options.add_argument('--disable-gpu')
driver1=webdriver.Firefox(firefox_options=options)

driver1.get("https://v6.bang.weibo.com/czv/caijing");

elems1 = driver.find_elements_by_tag_name("h3");
print(len(elems1))

20


下面的代码打印出排行榜上的微博名。

In [3]:
for elem in elems:
    print(elem.text)

龚凯杰
宇辉战舰
武汉刘正涛
侯宁
洪榕
何天恩
金融段子王
新视界发现者
唐史主任司马迁
战鹰gavin50
浙商投资卢献星V
短线刀客888
思多金
况秀猛
雨农观察
老虾扯蛋
张清华-价值投资最佳代言
遂昌快活林
李大霄
苏渝


注意到排行榜上的微博的简介在标签`<h4>`中，继续提取排行榜上微博的简介：

In [4]:
elems = driver.find_elements_by_tag_name("h4");
for elem in elems:
    print(elem.text)

音乐填词人 草根股评家，职业投资人 知名财经博主 微博签约自媒体
职业投资人 头条文章作者 微博签约自媒体
金通盛世投资有限公司 创始人、基金管理人，微博财经视频博主 头条文章作者 微博签约自媒体
独立财经观察家，时评家、社会学者、职业投资人 微博签约自媒体
职业投资人 知名财经博主
  衢州柯城奇正企业文化工作室 创始人 资讯视频自媒体
知名财经博主 头条文章作者 微博签约自媒体
澄泓财经首席成长股分析师 微博签约自媒体
职业投资人 微博签约自媒体
微博知名财经博主 鹰氏操盘学超话粉丝大咖 头条文章作者 微博签约自媒体
知名财经博主
微博股评师 微博签约自媒体
头条文章作者
Torrey Hills Technologies, LLC总裁
职业投资人 微博签约自媒体
职业投资人 微博签约自媒体
职业投资人 微博签约自媒体
浙江省遂昌金矿有限公司经济师，工程师 职业投资人 微博签约自媒体
英大证券首席经济学家 微博签约自媒体
炒股名博、独立财经观察家、学者、教授 微博签约自媒体


注意到排行榜上的微博的影响力分数在标签<span>中，但如果使用下面的代码来提取排行榜上微博的影响力分数，就会提取出很多无关的信息，因为这些无关的信息也放在<span>标签中。

In [5]:
elems = driver.find_elements_by_tag_name("span");
for elem in elems:
    print(elem.text)


98.48


我不问 + 你不说 = 误会 我问了 + 你不说 = 隔阂 我问了 + 你说了 = 尊重 你想说 + 我想问 = 默契 我不问 + 你说了 = 信任 心若亲近，言行必如流水般自然 心若疏远，言行只如三秋之树般萧瑟。 且行且珍惜，多一些沟通，多一份信任。 ——

97.98


调整如期而至，节前谨慎为上 #辉声辉色论涨停# #投资达人说#http://t.cn/EPMr0ZX（下载App->http://t.cn/RDUuslr）

97.41


#吴秀波老婆# 女演员陈昱霖发文透露自己和已婚演员吴秀波是恋人关系， 已有七年感情， 过去七年做牛做马在照顾吴秀波。 而波叔又上了的女演员张芷溪， 因由张芷溪的“骚扰”， 自己已患上抑郁症。 等下， 让俺整理一下思路， 波叔不是有老婆孩子了吗？ 俺明白了“小三”被“小四”了……。 说实话
04
97.10


股市跌的时候微博上很热闹，反弹了反而安静了许多！都在静静地过节吃月饼？
05
95.16


一个发达国家老大，一个发展中国家老大，两个老大出现了30年来最严重的“摩擦”，这是事实，但美国经济持续走好，经济晴雨表股市也持续创出新高，而中国股市却持续走低……不得不说这逻辑不通，一定有地方判断出现错误了。看中国股市情况投资者对未来经济极度悲观，这样一个经济体经济出现问题竟然对美
06
94.97


最新消息！！2小时前，帝吧百万大军横扫瑞典！第二波“血洗瑞典”已经完成！帝吧出征，寸草不生！对方已彻底投降，全线崩溃！所有的版面都已被占领！壮哉，我中华儿女！！[中国赞]
07
94.31


【最顾家贪污犯：贪污百万为原配治病，原配死后为新女友买房！】近日，北京一中院公开宣判了被告人王喜禄犯贪污罪一案。中国农业大学附属中学会计王喜禄，利用自己的职务便利，多次通过伪造外聘专家信息、虚增代课教师人数等手段，以发放工资的形式骗取该学校财政资金395万余元，后陆续用于其个人消费
08
94.00


【新视界每日划重点】证监会核发1家企业IPO批文筹资总额不超6亿；美股道指再创新高，香港恒生指数跌近2%；油价三个月内可能升破100美元/桶；银河电子：拟斥资0.3亿-3亿元回购公司股份；乾景园林：联合预中标4.5亿元项目；恒润股份：拟1.8亿元收购光科设备51%股权；鸿特科技：拟携手小黄狗环保科技，建
09
93

分析定位到影响力分数的Xpath发现，第一个影响力分数的Xpath是“/html/body/div[2]/div/div[2]/div[1]/div[2]/article/section[2]/div/div[1]/a/div/div/div[1]/div[1]/dl/dd/span[1]”，而第20个影响力分数的Xpath是“/html/body/div[2]/div/div[2]/div[1]/div[2]/article/section[2]/div/div[20]/a/div/div/div[1]/div[1]/dl/dd/span[1]”。Xpath中的“a/div/div/div[1]/div[1]/dl/dd/span[1]”是1-20名微博的影响力分数的Xpath后部所共有的，因此可以使用这个线索来提取影响力分数。

In [6]:
elems = driver.find_elements_by_xpath("//a/div/div/div[1]/div[1]/dl/dd/span[1]");
for elem in elems:
    print(elem.text)

98.48
97.98
97.41
97.10
95.16
94.97
94.31
94.00
93.67
93.63
93.56
93.54
93.53
93.38
93.16
92.72
92.70
92.68
92.66
92.35


## 抓取Ajax动态页面

AJAX即“Asynchronous Javascript And XML”（异步JavaScript和XML），是指一种创建交互式网页应用的网页开发技术。
AJAX = 异步 JavaScript和XML（标准通用标记语言的子集）。
AJAX 是一种用于创建快速动态网页的技术。
通过在后台与服务器进行少量数据交换，AJAX 可以使网页实现异步更新。这意味着可以在不重新加载整个网页的情况下，对网页的某部分进行更新。
传统的网页（不使用 AJAX）如果需要更新内容，必须重载整个网页页面。
上面的微博排行榜当中，我们只提取了排名1-20名的微博博主信息，想拿到21-40名的信息需要把页面往下拉，这样下面的信息才会显示。下面的代码实现了“把页面往下拉6000个像素”的效果。为什么是6000个像素，这是多次试验之后得到的数值。

In [7]:
driver.execute_script("window.scrollBy(0,6000)")

此时打印提取的`<h3>`标签的个数，发现可以提取出40个而不是原来的20个，说明第21-40名的信息也被提取出来了。

In [9]:
elems = driver.find_elements_by_tag_name("h3");
print(len(elems))

40


下面是打印出排名21-40名的微博的名称。

In [13]:
for i in range(20,40):
    print(elems[i].text)

金融八卦女
广闻博见V
股指期货if先生
石小杰微博
价值投资二十年
樊建川
余丰慧
林登万大人
谁是谁非任评说
炒短线的老李
魅仙儿炒股秘籍
A股侠女
花荣
天狼50陈浩
吴其伦
澄泓-研究君
智泉古道
李云飞-豪七的爸比
混沌与概率1997
卢麒元


下面是他们的简介。

In [14]:
elems = driver.find_elements_by_tag_name("h4");
for i in range(20,40):
    print(elems[i].text)

微博知名财经博主 头条文章作者 微博签约自媒体
知名财经博主 微博股评师 微博签约自媒体
职业投资人 知名财经博主 微博签约自媒体
中国煤矿文工团说唱团团长、著名相声演员
博众投资证券 讲师 知名财经博主 财经视频自媒体
四川省人大常委会委员，建川博物馆馆长 微博签约自媒体
经济金融专家；《互联网金融革命》作者 微博股评师 知名财经博主 微博签约自媒体
微博股评师 财经视频自媒体
中信改革发展研究院资深研究员、政法大学客座教授、投资人、律师 微博签约自媒体
知名财经博主 头条文章作者 微博签约自媒体
知名财经博主
职业投资人
股市职业操盘专家，知名财经作家花荣 微博签约自媒体
鼎信汇金（北京）投资管理有限公司总经理及首席策略师 头条文章作者
知名财经评论人 微博投资专家 微博签约自媒体
澄泓研究首席研究员 微博签约自媒体
知名财经博主 微博签约自媒体
北京中昆金信投资管理有限公司董事长 金信私募基金经理 头条文章作者 微博签约自媒体
福建宏祥享通投资有限公司 投资总监 微博签约自媒体
深圳市金宗信投资有限公司 董事 知名财经博主 微博签约自媒体


最后是他们的影响力分数。

In [15]:
elems = driver.find_elements_by_xpath("//a/div/div/div[1]/div[1]/dl/dd/span[1]");
for i in range(20,40):
    print(elems[i].text)

92.22
92.18
92.11
92.04
91.89
91.79
91.66
91.38
91.29
91.24
91.18
91.09
90.98
90.76
90.66
90.65
90.57
90.52
90.46
90.41


大功告成，关闭driver。

In [16]:
driver.close()